---
* Importing python modules

In [1]:
import configparser
from configparser import ConfigParser
from distutils.util import strtobool
from datetime import datetime
import regex as re
import os
from os import path
import numpy as np
import pandas as pd
import json
import math
import ast
import pymysql
from tqdm import trange, tqdm

print("-> Python modules imported successfully.")

-> Python modules imported successfully.


---
* Importing custom scripts

In [2]:
from scripts.mut_regall import *
from scripts.round_up import *
from scripts.metadata_cond import *
from scripts.get_output import *
from scripts.assign_metadata import *

print("-> Custom function imported successfully.")

-> Custom function imported successfully.


---
* Cheeking if tables_output, tables_raw and tables_processed exists

In [3]:
main_dirs = ["tables_output", "tables_raw", "tables_processed"]
mains_dirs_exists = [os.path.exists(i) for i in main_dirs]

for i,j in zip(main_dirs,mains_dirs_exists):
    if j == False:
        print("-> Missing {} folder.".format(i),"\n{} folder created.".format(i),"\n")
        os.mkdir(i)
    else:
        print("-> {} folder already exists.".format(i),"\n   continuing.".format(i),"\n")

-> tables_output folder already exists. 
   continuing. 

-> tables_raw folder already exists. 
   continuing. 

-> tables_processed folder already exists. 
   continuing. 



---
* Reading config file
* Configuring the MySQL variables
* Cheeking if the raw tables need to be imported
* if no -> continuing
* if yes -> downloading the missing MySQL tables from the ImmuneDB server

In [4]:
config_file = open("config.ini","r")
config_str = config_file.read()
config = configparser.ConfigParser(allow_no_value=True)
config.read_string(config_str)

print("-> config.ini was read successfully.")

-> config.ini was read successfully.


In [5]:
catg_mysql = ["mysql_"+i for i in config["mysql"]]

for i,j in zip(catg_mysql,config["mysql"]):
    values = i.split("_")
    globals()[i] = config[values[0]][j]

In [6]:
req_raw_paths = ["tables_raw\\{db}\\{db}.".format(db=mysql_db)+i+".csv" for i in mysql_tables.split(",")]

#cheeking if the tables exists
if sum([os.path.exists(i) for i in req_raw_paths]) < 3:
    print("\nMissing raw data tables. ", "Connecting to MySQL server via config credentials...")

    # connecting to mysql server
    try:
        conn = pymysql.connect(host = mysql_host,
                               port = int(mysql_port),
                               user = mysql_user,
                               password = mysql_pass,
                               db = mysql_db)
        
        if isinstance(conn,pymysql.connections.Connection):
            conn_status = "\nSuccessfully connected to MySQL server."
            print(conn_status)
        else:
            conn_status = "\nError while connecnting to mysql server, cheek config.ini..."
            raise TypeError(conn_status)
        
    except:
        conn_status = "\nError while connecnting to mysql server, cheek config.ini..."
        raise TypeError(conn_status)


    # making sure dir exists
    raw_path = "tables_raw\\{}".format(mysql_db)
    if path.exists(raw_path) == False:
        os.mkdir(raw_path)
        print("\nCreated database folder")
    else:
        print("\nDatabase folder already exists, continuing...")
    
    # importing the missing tables
    db_tables = mysql_tables.split(",")
    len_tables = len(db_tables)
    path_csv = raw_path
    
    num = 1
    for i in db_tables:
        print("\nImporting missing talbes, this process  may take a while... \n")
        table_name = "{db}.{table}".format(db=mysql_db, table=i)
        temp_count = "\n ({n}/{len})".format(n=num, len=len_tables)
        
        if path.exists(raw_path + "\\{t_name}.csv".format(t_name=table_name)):
            print(table_name+" already imported.",temp_count,"\n -----------")
        else:
            print("importing {t_name}".format(t_name=table_name))
            temp_df = pd.read_sql_query("SELECT * FROM {t_name};".format(t_name=table_name), conn)
            temp_df.to_csv(raw_path+"\\{t_name}.csv".format(t_name=table_name, table=i))
            print("Imported {t_name} succecfully.".format(t_name=table_name), temp_count ,"\n -----------")
        num += 1
        
    print("\nDone importing database tables.")

else:
    print("\nTalbes already imported, continuing...")


Talbes already imported, continuing...


---
* Loading the raw tables

In [7]:
t_names = mysql_tables.split(",")

with tqdm(t_names, desc="loading raw datasets",unit="dataset", initial=0) as pbar_loadraw:
    for tb in pbar_loadraw:
        globals()[tb] = pd.read_csv('tables_raw\\{db}\\{db}.{table}.csv'.format(db=mysql_db, table=tb)
                                    ,index_col=0)
        pbar_loadraw.set_postfix_str(tb)

loading raw datasets: 100%|██████████| 3/3 [00:11<00:00,  3.86s/dataset, sample_metadata]


---
* Creating tables_processed dir if needed
* Creating metadata dataframe if needed

In [8]:
path_processed_dir = "tables_processed\\{}".format(mysql_db)
if os.path.exists(path_processed_dir) == False:
    os.mkdir(path_processed_dir)
    print(path_processed_dir, "created.")
else:
    print(path_processed_dir, "already exists, conitnuing...")

tables_processed\covid_vaccine_new already exists, conitnuing...


In [9]:
path_metdadata_df = "tables_processed\\{db}\\{db}.sample_metadata_df.csv".format(db=mysql_db)

if os.path.exists(path_metdadata_df):
    print("{db}.sample_metadata_df.csv already created, continuing...".format(db=mysql_db))
    metadata_df = pd.read_csv(path_metdadata_df, index_col=0)
                                                                                 
else:
    print("Creating {db}.sample_metadata_df.csv...".format(db=mysql_db))
   
    metadata_keys_og = mysql_metadata_og.split(",")
    metadata_keys_new =  mysql_metadata_new.split(",")
    meta_dict = dict(zip(metadata_keys_og, metadata_keys_new))

    metadata_og = sample_metadata[sample_metadata["key"].isin(metadata_keys_og)]
    metadata_og = metadata_og.replace({"key":meta_dict})

    sample_ids = np.sort(metadata_og["sample_id"].unique())
    metadata_df = pd.DataFrame({"sample_id":sample_ids})
    metadata_df[metadata_keys_new] = np.nan

    for i in sample_ids:
        temp_sid = i
        for j in metadata_keys_new:
            cond_sid = (metadata_og["sample_id"] == i)
            cond_key = (metadata_og["key"] == j)
            metadata_df.loc[metadata_df["sample_id"]==i,j] = metadata_og.loc[(metadata_og["sample_id"]==i)&(metadata_og["key"]==j),"value"].values
    metadata_df.to_csv(path_metdadata_df)
    print("Done.")

Creating covid_vaccine_new.sample_metadata_df.csv...
Done.


C:\Users\Daniel\AppData\Local\Temp\ipykernel_24156\2775119400.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['Non-Spike B']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  metadata_df.loc[metadata_df["sample_id"]==i,j] = metadata_og.loc[(metadata_og["sample_id"]==i)&(metadata_og["key"]==j),"value"].values
C:\Users\Daniel\AppData\Local\Temp\ipykernel_24156\2775119400.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['1 baseline']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  metadata_df.loc[metadata_df["sample_id"]==i,j] = metadata_og.loc[(metadata_og["sample_id"]==i)&(metadata_og["key"]==j),"value"].values


---
* Verifing that all the samples id's from metadata file exists in the clone_stats file.

In [10]:
metalist_sids = np.sort(metadata_df.sample_id.unique())
clones_sids = np.sort(clone_stats.dropna().sample_id.unique()).astype("int")

values_missing = np.setdiff1d(clones_sids, metalist_sids)
values_common = np.intersect1d(metalist_sids, clones_sids)

if len(values_missing) > 0:
    print("missing sample_id from metadata file:",values_missing)
    clone_stats = clone_stats[clone_stats["sample_id"].isin(values_common)]
    raise TypeError("verify metadata sample_id values") 

else:
    print("No missing sample_id values.")

No missing sample_id values.


---
* loading clones_merged if exists, if not creating and saving
* custom function that extract mutations infromation from the "mutation" json in each row
* Adding the germline infromation to the clone_stats df
* Dropping null sample_id rows (cannot assign metadata for those rows)
* converting "sample_id" values to int instead of floats
* assiging the metadata into the merged table (applying assign_metadata)
* renaming id_x to id after merging (left had "id" colum while right had "id"=="clone_id")
* reseting the index

In [11]:
path_clones_merged = "tables_processed\\{db}\\{db}.clones_merged.csv".format(db=mysql_db)

if os.path.exists(path_clones_merged):
    clones_merged = pd.read_csv(path_clones_merged)
    print("clones_merged dataframe exists, loading and continuing...")

else:
    print("Creating clones_merged.csv")
    clones_merged = clone_stats.merge(right=clones[["id","germline"]],
                                      how="left",
                                      left_on="clone_id",
                                      right_on="id")    
    
    clones_merged = clones_merged[clones_merged["sample_id"].notnull()]        
    clones_merged[list(metadata_df.columns)[1:]] = list(clones_merged["sample_id"].apply(assign_metadata, args=(metadata_df,)))
    clones_merged.rename({"id_x":"id"},axis="columns",inplace=True)
    clones_merged.reset_index(drop=True, inplace=True)
    clones_merged.to_csv(path_clones_merged)
    print("clones_merged dataframe created and saved, continuing...")

Creating clones_merged.csv
clones_merged dataframe created and saved, continuing...


---
* Creating df with the relevent mutations infromation for each clone
* Cleaning the df and adding relevent data
* Saving the df

In [12]:
path_mut_df = "tables_processed\\{db}\\{db}.mut_df.csv".format(db=mysql_db)

if os.path.exists(path_mut_df):
    mut_df = pd.read_csv(path_mut_df,index_col=0)
    print("mut_df dataframe exists, loading and continuing....")

else:           
    clones_merged["regions_all"] = clones_merged["mutations"].apply(mut_regall)
    clones_raval = clones_merged.copy()
    ra_val = []

    with tqdm(range(0,len(clones_raval))) as tqdm_reval:
        for i in tqdm_reval:
            length = len(clones_raval.loc[i,"regions_all"]) # length of the list, number of mutations is the colum
            value = clones_raval.loc[i,"regions_all"] # the value mutations themselves list of lists/ list / np.nan
            id_value = clones_raval.loc[i,"id"] # id value of the row
            clone_id = clones_raval.loc[i,"clone_id"] # clone_id value of the row
            subject_id = clones_raval.loc[i,"subject_id"]# subject_id value of the row
            sample_id = clones_raval.loc[i,"sample_id"] # sample_id value of the row
            funct = clones_raval.loc[i,"functional"] # functional value of the clone
            total_cnt = clones_raval.loc[i,"total_cnt"] # target of the antibody
            unique_cnt = clones_raval.loc[i,"unique_cnt"] # unique sequence is clone
            germline = clones_raval.loc[i,"germline"] #germline sequence
            top_seq = clones_raval.loc[i,"top_copy_seq_sequence"] #top copy of sequence
            metadata = clones_raval.loc[i,metadata_df.columns[1:]].values.flatten().tolist() #metadata list value
            ins_val = [id_value, clone_id, subject_id, sample_id, funct, total_cnt,unique_cnt, germline, top_seq] + metadata
            
            # if single row of mutation
            if length == 1:
                to_aas = value[0][4].replace(" ","").replace("''","").split(",")
                
                if (len(to_aas) == 1):
                    temp_list = list(value[0])
                    ra_val.append(ins_val + temp_list) 
                    
                else:
                    for i in range(0,len(to_aas)):
                        temp_list = list(value[0])
                        temp_list[4] = to_aas[i]
                        ra_val.append(ins_val + temp_list)
            
            # if multiple rows of mutations
            if length > 1:
                for j in range(0,length):
                        sub_value = list(value[j]) #each row
                        temp_list = sub_value
                        
                        # making sure that the length of the list is correct, in some rows there is missing values
                        if len(sub_value) == 8:
                            to_aas = sub_value[4].replace(" ","").replace("'","").split(",")
                            
                            if len(to_aas) == 1:
                                ra_val.append(ins_val + temp_list)
                            elif len(to_aas) > 1:
                                for aa in set(to_aas): # set() removes duplicate values
                                    temp_list[4] = aa
                                    ra_val.append(ins_val + temp_list)
                                             
            elif length == 0:
                ra_val.append(ins_val + np.full(shape=len(value), fill_value=np.nan).tolist())
        
        mut_df_cols = ["id", "clone_id", "subject_id", "sample_id", "functional", "total_cnt","unique_cnt", "germline", "top_seq"]
        mut_info_cols = ["pos","from_nt","from_aa","to_nt","to_aas","unique","total","intermidiate_aa"]
        
        mut_df = pd.DataFrame(data=ra_val, columns = mut_df_cols + metadata_df.columns[1:].tolist() + mut_info_cols)
        mut_df["to_aas"] = mut_df["to_aas"].str.replace("'","") #cleaning to_aas string
        mut_df.replace({"to_aas":{"None":np.nan}}, inplace=True) #turining none values to np.nan
        mut_df.dropna(axis=0,subset=["pos","to_aas"], ignore_index=True, inplace=True) #dropping null rows of "pos" and "to_aas"
        
        mut_df.insert(6,"pos_aa",np.nan) #inserting amino acid position column
        mut_df.insert(6,"pos_nt",np.nan) #inserting nucleotide position column
        mut_df.loc[:,"pos_nt"] = mut_df.loc[:,"pos"].apply(int)+1 #filling the pos_nt column
        mut_df.loc[:,"pos_aa"] = ((mut_df.loc[:,"pos_nt"])/3).apply(round_up) #fillint the pos_aa column 
        mut_df.astype({"pos_nt":"int","pos_aa":"int"})
        mut_df.drop(axis=1,columns="pos",inplace=True) #dropping the og column (it was -1 in position...)
        mut_df["syn"] = (mut_df["from_aa"] == mut_df["to_aas"]).apply(int) #creating syn column
    
        mut_df.to_csv(path_mut_df)
        print("{}.mut_df dataframe created and saved, continuing....".format(mysql_db))

100%|██████████| 85841/85841 [00:22<00:00, 3775.37it/s]


covid_vaccine_new.mut_df dataframe created and saved, continuing....


---
* Importing amino acid type dictionary.
* Creating function that will cat string of position.before_aa_type.after_aa_type.
* Assigining output to new colum.

In [13]:
path = "scripts\\aa_type_dict.py"

with open(path, "r") as type_raw:
    type_str = type_raw.read()
    type_dict = json.loads(type_str)
    type_raw.close()

type_dict_T = {vi: k  for k, v in type_dict.items() for vi in v}

get_change = (lambda df,idict,paa,faa,taa: ".".join([str(int(df[paa])), idict[df[faa]], idict[df[taa]]]))

mut_df["from_type"] = mut_df["from_aa"].apply((lambda x,idict: idict[x]), idict=type_dict_T)
mut_df["to_type"] = mut_df["to_aas"].apply((lambda x,idict: idict[x]), idict=type_dict_T)
mut_df["type_change"] =  mut_df.apply(get_change, 
                                      idict = type_dict_T,
                                      paa = "pos_aa",
                                      faa = "from_aa",
                                      taa = "to_aas",
                                      axis = 1)

---
* Filter unwated mutations.

In [14]:
filt_pos_aa = (mut_df["pos_aa"].between(1,104,inclusive='both')) # from aa positions 1->104
filt_functional = (mut_df["functional"] == 1) # only functional clones
filt_unique_cnt = (mut_df["unique_cnt"] > 1) # only clones with more than 1 unique sequence
filt_syn = (mut_df["syn"] == 0) # only non-syn mutations (substitutions)

filt_mut_df = mut_df[filt_pos_aa & filt_functional & filt_syn & filt_unique_cnt]

if bool(strtobool(mysql_filter_trunk)):
    filt_mut_df["mut_freq"] = filt_mut_df["unique"]/filt_mut_df["unique_cnt"]
    filt_mut_df = filt_mut_df[(filt_mut_df["mut_freq"] >= float(mysql_trunk_threshold)) & 
    (filt_mut_df["unique_cnt"] > int(mysql_nclones_threshold))]
    print("-> applied fitlers")
   
filt_mut_df["subject_id"] = filt_mut_df["subject_id"].apply(str)

print("-> Filtred mutations rows, conitnuing...")

-> Filtred mutations rows, conitnuing...


C:\Users\Daniel\AppData\Local\Temp\ipykernel_24156\3471186527.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filt_mut_df["subject_id"] = filt_mut_df["subject_id"].apply(str)


---
* Changing order of sorting if needed.

In [15]:
# Changing the order of the label presentation
change_order = bool(strtobool(mysql_order_change))

catagories_dict = {}

if change_order:
    val_list = mysql_order_newval.split(",")
    
    for i in val_list:
        temp_val_list = list(filt_mut_df[i].unique())
        temp_val_list.sort()
        catagories_dict[i] = temp_val_list

else:
    unique_subj = list(filt_mut_df["subject_id"].unique())
    
    catagories_dict["subject_id"] = unique_subj
    val_list = list(metadata_df.columns[1:])
    val_list.insert(0,"subject_id")
    
    for i in val_list:
        catagories_dict[i] = np.sort(filt_mut_df[i].unique())  

print("-> change order: {}".format(change_order), "\n-> Order: {}".format(list(catagories_dict.keys())))

-> change order: False 
-> Order: ['subject_id', 'ab_target', 'time_point']


---
* Remapping values if needed.

In [16]:
# values for metadata remap
remap_subject_id = ["subj_"+i for i in np.sort(filt_mut_df["subject_id"].unique())]
remap_time_point = [i[0] for i in np.sort(filt_mut_df["time_point"].unique())]
remap_ab_type = ["sp" if i == "Spike+ Mem B" else "sn" for i in np.sort(filt_mut_df["ab_target"].unique())]

In [17]:
remap = bool(strtobool(mysql_remap))

if remap:
    custom_list = [remap_subject_id, 
                   remap_time_point,
                   remap_ab_type]

    remap_vals = {old_val:new_val for old_dic,new_dic in zip(catagories_dict.values(),custom_list) for old_val,new_val in zip(old_dic,new_dic)}
    catagories_dict = {i:j for i,j in zip(list(catagories_dict.keys()),custom_list)}
    
   
    filt_mut_df.loc[:,val_list] = filt_mut_df.loc[:,val_list].replace(remap_vals)
    catagories_dict = {i:j for i,j in zip(list(catagories_dict.keys()),[i for i in catagories_dict.values()])}

---
* Creating & saving the metadata table

In [18]:
cond_matrix = metadata_cond(catagories_dict).astype("str")
cond_matrix.to_csv("tables_processed\\{db}\\{db}.metadata_matrix.csv".format(db=config["mysql"]["db"]))

print("-> Saved metadata matrix.")

-> Saved metadata matrix.


---
[https://www.imgt.org/IMGTScientificChart/Numbering/IMGTIGVLsuperfamily.html](https://www.imgt.org/IMGTScientificChart/Numbering/IMGTIGVLsuperfamily.html)

* Choosing the analysis aa positions range.
* Creating frequencies of subtition mutations for LPA analysis.

In [19]:
range_all = range(1,105) #FR1, CDR1, FR2, CDR2, FR3
#range_cdr = list(range(27,39)) + list(range(56,66)) #CDR1, CDR2
#range_fr = list(range(1,27))+list(range(39,56))+list(range(66,105)) #FR1,FR2,FR3

df_input = filt_mut_df.copy()
pos_range = range_all
output_summerylis = []
df_output = pd.DataFrame({"pos_aa":pos_range})

for i in tqdm(range(0,len(cond_matrix))):
    df_temp = df_input.copy()
    temp_vals = cond_matrix.iloc[i,:].values

    for name, val in zip(val_list,temp_vals): 
        df_temp = df_temp[df_temp[name] == val]

    col_name = ".".join(temp_vals)
    output_summerylis.append([col_name,len(df_temp["clone_id"].unique())])
    
    df_getoutput = get_output(df_temp,col_name,pos_range,method="nunique_count")
    df_output = df_output.merge(df_getoutput, on="pos_aa", how="left")

if os.path.exists("tables_processed\\{db}".format(db=mysql_db)) == False:
    os.mkdir("tables_processed\\{db}".format(db=mysql_db))

df_output.to_csv("tables_processed\\{db}\\{db}.{val}-documents_substitution_tfreq.csv".format(db=mysql_db,val=".".join(val_list)))

df_domain = get_output(df_input,"domain",pos_range,method="nunique_weight")
df_domain.to_csv("tables_processed\\{db}\\{db}.{val}-domain_substitution_tfreq.csv".format(db=mysql_db,val=".".join(val_list)))

print("-> documents frequencies saved.","\n-> Domain frequenceis saved.")

100%|██████████| 30/30 [00:04<00:00,  6.61it/s]


-> documents frequencies saved. 
-> Domain frequenceis saved.


---
* Creating domain output file.
* Creating documents output file.

In [20]:
path_output = "tables_output\\{}".format(mysql_db)
if os.path.exists(path_output) == False:
    os.mkdir(path_output)
    print("-> {} folder created.".format(path_output))

output_domain = df_domain.rename({"pos_aa":"element", "domain":"global_weight"},axis=1)

for i in tqdm(df_output.columns[1:]):
    document = i
    elements =  df_output["pos_aa"]
    frequency = df_output[i]
    temp_df = pd.DataFrame({"document":i, 
                            "element":elements, 
                            "frequency_in_document":frequency})

    if i == df_output.columns[1]:
        output_documents = temp_df
    else:
        output_documents = pd.concat([output_documents, temp_df])

for i,j in zip([output_domain, output_documents],["domain","documents"]):
    i.to_csv(path_output+"\\{db}.{type}.csv".format(db=mysql_db, type=j))

print("-> output files saved.")

100%|██████████| 30/30 [00:00<00:00, 2210.24it/s]

-> output files saved.


---
*

In [21]:
path_output = "tables_output\\{}".format(mysql_db)

for i in tqdm(range(0,len(cond_matrix))):
    df_temp = df_input.copy()
    temp_vals = cond_matrix.iloc[i,:].values

    for name, val in zip(val_list,temp_vals): 
        df_temp = df_temp[df_temp[name] == val]

    col_name = ".".join(temp_vals)

    out_df = df_temp.groupby("type_change").agg({"clone_id":"nunique"}).rename({"clone_id":"freq"},axis=1).reset_index()
    temp_freq = pd.DataFrame({"document":col_name, "element":out_df["type_change"], "frequency_in_document":out_df["freq"]})
    temp_freq["pos"] = temp_freq["element"].apply(lambda x: int(x.split(".")[0]))
    temp_freq = temp_freq.sort_values("pos").reset_index(drop=True)
    temp_freq = temp_freq.iloc[:,0:-1]

    if i == 0:
        changetype_df = temp_freq

    else:
        changetype_df = pd.concat([changetype_df, temp_freq], axis=0)

    changetype_df.to_csv(path_output+"\\{db}.{type}.csv".format(db=mysql_db, type="change_type"))

100%|██████████| 30/30 [00:05<00:00,  5.96it/s]


In [22]:
df_input["dataset"] = df_input["ab_target"] + "." + df_input["time_point"] + "." + df_input["subject_id"]
#df_input["dataset"] = df_input["subject_id"] + "." +df_input["ab_target"]

sizes= pd.DataFrame(df_input.groupby(["dataset","pos_aa"]).size().reset_index())
sizes.rename(columns={0:"n_mut"},inplace=True)
sizes.to_csv(path_output+"\\{db}.{type}.csv".format(db=mysql_db, type="dataset_sizes"))

---

In [23]:
output_documents

,document,element,frequency_in_document
0,subj_3.1.sn,1,0.0
1,subj_3.1.sn,2,0.0
2,subj_3.1.sn,3,0.0
3,subj_3.1.sn,4,0.0
4,subj_3.1.sn,5,0.0
...,...,...,...
99,subj_7.4.sp,100,0.0
100,subj_7.4.sp,101,0.0
101,subj_7.4.sp,102,0.0
102,subj_7.4.sp,103,0.0
